In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb

import gzip

import sklearn.preprocessing

from scipy.spatial.distance import cosine


In [2]:
# We split trainfile into train and valid pretty early on
trainfile_orig = pd.read_csv('train.csv')
trainfile = trainfile_orig.copy()

test_orig = pd.read_csv('test.csv')
test = test_orig.copy()

sub_orig = pd.read_csv('sample_submission.csv')
sub = sub_orig.copy()

In [3]:
def chop_triptype(tt):
    return int(tt[9:])

In [4]:
sub.loc[0].keys()

Index([u'VisitNumber', u'TripType_3', u'TripType_4', u'TripType_5',
       u'TripType_6', u'TripType_7', u'TripType_8', u'TripType_9',
       u'TripType_12', u'TripType_14', u'TripType_15', u'TripType_18',
       u'TripType_19', u'TripType_20', u'TripType_21', u'TripType_22',
       u'TripType_23', u'TripType_24', u'TripType_25', u'TripType_26',
       u'TripType_27', u'TripType_28', u'TripType_29', u'TripType_30',
       u'TripType_31', u'TripType_32', u'TripType_33', u'TripType_34',
       u'TripType_35', u'TripType_36', u'TripType_37', u'TripType_38',
       u'TripType_39', u'TripType_40', u'TripType_41', u'TripType_42',
       u'TripType_43', u'TripType_44', u'TripType_999'],
      dtype='object')

In [5]:
triptype_map = {}
triptypestr_map = {}

c = 0
for k in sub.loc[0].keys():
    if 'TripType_' in k:
        triptypestr_map[k] = c
        print triptypestr_map
        triptype_map[chop_triptype(k)] = c
        print triptype_map     
        c += 1
        print c

{'TripType_3': 0}
{3: 0}
1
{'TripType_4': 1, 'TripType_3': 0}
{3: 0, 4: 1}
2
{'TripType_4': 1, 'TripType_5': 2, 'TripType_3': 0}
{3: 0, 4: 1, 5: 2}
3
{'TripType_6': 3, 'TripType_4': 1, 'TripType_5': 2, 'TripType_3': 0}
{3: 0, 4: 1, 5: 2, 6: 3}
4
{'TripType_6': 3, 'TripType_7': 4, 'TripType_4': 1, 'TripType_5': 2, 'TripType_3': 0}
{3: 0, 4: 1, 5: 2, 6: 3, 7: 4}
5
{'TripType_8': 5, 'TripType_6': 3, 'TripType_7': 4, 'TripType_4': 1, 'TripType_5': 2, 'TripType_3': 0}
{3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5}
6
{'TripType_8': 5, 'TripType_9': 6, 'TripType_6': 3, 'TripType_7': 4, 'TripType_4': 1, 'TripType_5': 2, 'TripType_3': 0}
{3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6}
7
{'TripType_12': 7, 'TripType_8': 5, 'TripType_9': 6, 'TripType_6': 3, 'TripType_7': 4, 'TripType_4': 1, 'TripType_5': 2, 'TripType_3': 0}
{3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6, 12: 7}
8
{'TripType_14': 8, 'TripType_12': 7, 'TripType_8': 5, 'TripType_9': 6, 'TripType_6': 3, 'TripType_7': 4, 'TripType_4': 1, 'TripType_5': 2,

In [6]:
triptype_map[45] = triptype_map[999]

print(triptypestr_map)
print(triptype_map)

{'TripType_38': 30, 'TripType_39': 31, 'TripType_36': 28, 'TripType_37': 29, 'TripType_34': 26, 'TripType_35': 27, 'TripType_32': 24, 'TripType_33': 25, 'TripType_30': 22, 'TripType_31': 23, 'TripType_14': 8, 'TripType_15': 9, 'TripType_12': 7, 'TripType_18': 10, 'TripType_19': 11, 'TripType_999': 37, 'TripType_29': 21, 'TripType_28': 20, 'TripType_21': 13, 'TripType_20': 12, 'TripType_23': 15, 'TripType_22': 14, 'TripType_25': 17, 'TripType_24': 16, 'TripType_27': 19, 'TripType_26': 18, 'TripType_43': 35, 'TripType_42': 34, 'TripType_41': 33, 'TripType_40': 32, 'TripType_44': 36, 'TripType_8': 5, 'TripType_9': 6, 'TripType_6': 3, 'TripType_7': 4, 'TripType_4': 1, 'TripType_5': 2, 'TripType_3': 0}
{3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6, 12: 7, 14: 8, 15: 9, 18: 10, 19: 11, 20: 12, 21: 13, 22: 14, 23: 15, 24: 16, 25: 17, 26: 18, 27: 19, 28: 20, 29: 21, 30: 22, 31: 23, 32: 24, 33: 25, 34: 26, 35: 27, 36: 28, 37: 29, 38: 30, 39: 31, 40: 32, 41: 33, 42: 34, 43: 35, 44: 36, 45: 37, 999: 

In [7]:
# replace days of week.  Yes this could be defined with one line...

dow = {}
dow['Monday'] = 1
dow['Tuesday'] = 2
dow['Wednesday'] = 3
dow['Thursday'] = 4
dow['Friday'] = 5
dow['Saturday'] = 6
dow['Sunday'] = 7

In [8]:
def preproc(df):
    df.Weekday.replace(dow, inplace=True)
    
    df.FinelineNumber.replace({np.nan: -1}, inplace=True)

    df.FinelineNumber = df.FinelineNumber.astype(int)
    df.FinelineNumber += 1

    # there are nan's that need to get converted
    df.DepartmentDescription = df.DepartmentDescription.astype(str)
    
    if 'TripType' in df:
        enc = sklearn.preprocessing.OneHotEncoder()
        enc.fit(df.TripType.reshape(-1, 1))
    
        df.TripTypeOneHot = enc.transform(df.TripType.reshape(-1, 1)).toarray()

In [9]:
# replace type 999 with 45 to keep from wasting memory in structs
#trainfile.TripType.replace({999: 45}, inplace=True)

trainfile.TripType.replace(triptype_map, inplace=True)

preproc(trainfile)
preproc(test)

In [10]:
trainfile

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,37,5,5,68113152929,-1,FINANCIAL SERVICES,1001
1,22,7,5,60538815980,1,SHOES,8932
2,22,7,5,7410811099,1,PERSONAL CARE,4505
3,18,8,5,2238403510,2,PAINT AND ACCESSORIES,3566
4,18,8,5,2006613744,2,PAINT AND ACCESSORIES,1018
5,18,8,5,2006618783,2,PAINT AND ACCESSORIES,1018
6,18,8,5,2006613743,1,PAINT AND ACCESSORIES,1018
7,18,8,5,7004802737,1,PAINT AND ACCESSORIES,2803
8,18,8,5,2238495318,1,PAINT AND ACCESSORIES,4502
9,18,8,5,2238400200,-1,PAINT AND ACCESSORIES,3566


In [11]:
# Now split the train data into train+valid.  2.5% with seed 0 appears to be pretty good, at least until this
# gets competitive

visits = np.unique(trainfile.VisitNumber)

# determine visits to go in validation set
np.random.seed(0)
validation_visits = np.random.choice(visits, int(len(visits) * .025))

# compute (raw) rows in validation set
validation_set = (trainfile.VisitNumber == validation_visits[0])

for i in range(1, len(validation_visits)):
    validation_set |= trainfile.VisitNumber == validation_visits[i]

valid = trainfile.ix[validation_set]
    
# flip that around to get the train set
training_set = np.invert(validation_set)
train = trainfile.ix[training_set]

In [12]:
train

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,37,5,5,68113152929,-1,FINANCIAL SERVICES,1001
1,22,7,5,60538815980,1,SHOES,8932
2,22,7,5,7410811099,1,PERSONAL CARE,4505
3,18,8,5,2238403510,2,PAINT AND ACCESSORIES,3566
4,18,8,5,2006613744,2,PAINT AND ACCESSORIES,1018
5,18,8,5,2006618783,2,PAINT AND ACCESSORIES,1018
6,18,8,5,2006613743,1,PAINT AND ACCESSORIES,1018
7,18,8,5,7004802737,1,PAINT AND ACCESSORIES,2803
8,18,8,5,2238495318,1,PAINT AND ACCESSORIES,4502
9,18,8,5,2238400200,-1,PAINT AND ACCESSORIES,3566


In [13]:
# mclogloss.py (todo: copy in original link)
def multiclass_log_loss(y_true, y_pred, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    https://www.kaggle.com/wiki/MultiClassLogLoss

    idea from this post:
    http://www.kaggle.com/c/emc-data-science/forums/t/2149/is-anyone-noticing-difference-betwen-validation-and-leaderboard-error/12209#post12209

    Parameters
    ----------
    y_true : array, shape = [n_samples]
    y_pred : array, shape = [n_samples, n_classes]

    Returns
    -------
    loss : float
    """
    predictions = np.clip(y_pred, eps, 1 - eps)

    # normalize row sums to 1
    predictions /= predictions.sum(axis=1)[:, np.newaxis]

    actual = np.zeros(y_pred.shape)
    rows = actual.shape[0]
    actual[np.arange(rows), y_true.astype(int)] = 1
    vsota = np.sum(actual * np.log(predictions))
    return -1.0 / rows * vsota


In [14]:
def build_visitmaps(df):
    visits = np.unique(df.VisitNumber)
    visitmap = {}

    vtype = np.zeros(len(visits))

    if 'TripType' in df:
        for i in range(0, len(visits)):
            vtype[i] = df.ix[df.VisitNumber == visits[i]].iloc[0].TripType
    
    for i in range(0, len(visits)):
        visitmap[visits[i]] = i
        
    return visits, vtype, visitmap

In [15]:
valid_visits, valid_type, valid_visitmap = build_visitmaps(valid)
test_visits, test_type, test_visitmap = build_visitmaps(test)

In [16]:
valid_visitmap

{155648: 1900,
 106498: 1294,
 106501: 1295,
 40967: 488,
 98313: 1199,
 16394: 204,
 24587: 293,
 73742: 903,
 139283: 1694,
 114708: 1394,
 16405: 205,
 114710: 1395,
 147485: 1811,
 163871: 2001,
 188450: 2321,
 113330: 1386,
 57390: 700,
 163887: 2002,
 184328: 2258,
 49202: 601,
 49207: 602,
 49211: 603,
 114748: 1396,
 188426: 2320,
 90174: 1108,
 16452: 206,
 41050: 489,
 65627: 788,
 146106: 1791,
 139358: 1695,
 8287: 105,
 122980: 1502,
 32869: 393,
 180329: 2204,
 180330: 2205,
 65653: 789,
 73847: 905,
 16504: 207,
 172155: 2107,
 131198: 1591,
 125757: 1531,
 24711: 294,
 98440: 1200,
 131210: 1592,
 73868: 906,
 24717: 295,
 155804: 1901,
 188573: 2324,
 24740: 296,
 180389: 2206,
 90279: 1109,
 65704: 790,
 131241: 1593,
 106667: 1296,
 155820: 1902,
 164016: 2003,
 90290: 1110,
 183043: 2245,
 174795: 2138,
 131270: 1594,
 199: 0,
 8396: 106,
 65742: 791,
 180432: 2207,
 24788: 297,
 214: 1,
 65754: 792,
 139486: 1697,
 164063: 2004,
 137936: 1679,
 49386: 604,
 73965: 

In [17]:
valid_type

array([  4.,  34.,  32., ...,  36.,  36.,  32.])

In [18]:
# make categorical #'s.  TODO: move up to preproc???
ddcat = np.unique(train.DepartmentDescription)

# build map
ddmap = {}
for i in range(0, len(ddcat)):
    ddmap[ddcat[i]] = i
           
# apply map
def makeddint(df):
    ddcatout = np.zeros(len(df.DepartmentDescription))
    for i in range(0, len(ddcat)):
        ddcatout[np.where(df.DepartmentDescription == ddcat[i])] = i
    
    # XXX: this is raising a SettingWithCopyWarning on valid and possibly train
    df['DepartmentDescriptionInt'] = ddcatout.astype(int)
    
    return df

In [19]:
train = makeddint(train)
valid = makeddint(valid)
test = makeddint(test)

/home/deepu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
train

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,DepartmentDescriptionInt
0,37,5,5,68113152929,-1,FINANCIAL SERVICES,1001,20
1,22,7,5,60538815980,1,SHOES,8932,62
2,22,7,5,7410811099,1,PERSONAL CARE,4505,50
3,18,8,5,2238403510,2,PAINT AND ACCESSORIES,3566,49
4,18,8,5,2006613744,2,PAINT AND ACCESSORIES,1018,49
5,18,8,5,2006618783,2,PAINT AND ACCESSORIES,1018,49
6,18,8,5,2006613743,1,PAINT AND ACCESSORIES,1018,49
7,18,8,5,7004802737,1,PAINT AND ACCESSORIES,2803,49
8,18,8,5,2238495318,1,PAINT AND ACCESSORIES,4502,49
9,18,8,5,2238400200,-1,PAINT AND ACCESSORIES,3566,49


In [21]:
train_psc = train[train.ScanCount >= 1]
train_psc

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,DepartmentDescriptionInt
1,22,7,5,60538815980,1,SHOES,8932,62
2,22,7,5,7410811099,1,PERSONAL CARE,4505,50
3,18,8,5,2238403510,2,PAINT AND ACCESSORIES,3566,49
4,18,8,5,2006613744,2,PAINT AND ACCESSORIES,1018,49
5,18,8,5,2006618783,2,PAINT AND ACCESSORIES,1018,49
6,18,8,5,2006613743,1,PAINT AND ACCESSORIES,1018,49
7,18,8,5,7004802737,1,PAINT AND ACCESSORIES,2803,49
8,18,8,5,2238495318,1,PAINT AND ACCESSORIES,4502,49
10,18,8,5,5200010239,1,DSD GROCERY,4607,17
11,18,8,5,88679300501,2,PAINT AND ACCESSORIES,3505,49


In [22]:
# Construct set of usable DepartmentDescriptionInt keys (with >1000 per dept)
vc= train_psc.DepartmentDescriptionInt.value_counts()
vc

24    68088
17    65813
57    49214
16    42400
50    40250
32    27510
30    23875
52    22247
21    20484
31    15645
13    14745
5     14088
43    11338
34    10005
60     9747
11     9464
20     9343
51     9315
41     9168
56     8774
12     8232
37     7735
15     7033
3      6881
45     6218
28     5885
33     5748
66     5651
62     5648
2      4966
      ...  
53     2786
29     2724
18     2660
35     2347
63     2183
6      2071
58     1866
42     1841
9      1818
67     1666
49     1583
1      1216
68     1141
7       936
36      745
65      710
54      639
46      533
55      516
22      434
61      399
0       352
44      303
10      193
47       85
38       76
14       33
48       29
59       26
26        1
Name: DepartmentDescriptionInt, dtype: int64

In [23]:
for i in vc.iteritems():
    print i

(24, 68088)
(17, 65813)
(57, 49214)
(16, 42400)
(50, 40250)
(32, 27510)
(30, 23875)
(52, 22247)
(21, 20484)
(31, 15645)
(13, 14745)
(5, 14088)
(43, 11338)
(34, 10005)
(60, 9747)
(11, 9464)
(20, 9343)
(51, 9315)
(41, 9168)
(56, 8774)
(12, 8232)
(37, 7735)
(15, 7033)
(3, 6881)
(45, 6218)
(28, 5885)
(33, 5748)
(66, 5651)
(62, 5648)
(2, 4966)
(39, 4898)
(19, 4747)
(64, 4672)
(25, 4482)
(4, 4276)
(23, 4241)
(27, 3800)
(40, 3691)
(8, 3618)
(53, 2786)
(29, 2724)
(18, 2660)
(35, 2347)
(63, 2183)
(6, 2071)
(58, 1866)
(42, 1841)
(9, 1818)
(67, 1666)
(49, 1583)
(1, 1216)
(68, 1141)
(7, 936)
(36, 745)
(65, 710)
(54, 639)
(46, 533)
(55, 516)
(22, 434)
(61, 399)
(0, 352)
(44, 303)
(10, 193)
(47, 85)
(38, 76)
(14, 33)
(48, 29)
(59, 26)
(26, 1)


In [24]:

ddi_len = np.zeros(len(ddcat))
ddi_keys = {}

for i in vc.iteritems():
    ddi_len[i[0]] = i[1]
    
    if i[1] > 1000:
        ddi_keys[i[0]] = True

In [25]:
train_psc = train[train.ScanCount >= 1]
subsetted = np.full(len(train_psc), False, dtype=bool)

In [26]:
train_psc

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,DepartmentDescriptionInt
1,22,7,5,60538815980,1,SHOES,8932,62
2,22,7,5,7410811099,1,PERSONAL CARE,4505,50
3,18,8,5,2238403510,2,PAINT AND ACCESSORIES,3566,49
4,18,8,5,2006613744,2,PAINT AND ACCESSORIES,1018,49
5,18,8,5,2006618783,2,PAINT AND ACCESSORIES,1018,49
6,18,8,5,2006613743,1,PAINT AND ACCESSORIES,1018,49
7,18,8,5,7004802737,1,PAINT AND ACCESSORIES,2803,49
8,18,8,5,2238495318,1,PAINT AND ACCESSORIES,4502,49
10,18,8,5,5200010239,1,DSD GROCERY,4607,17
11,18,8,5,88679300501,2,PAINT AND ACCESSORIES,3505,49


In [27]:
enc = sklearn.preprocessing.OneHotEncoder()
enc.fit(train_psc.TripType.reshape(-1, 1))

OneHotEncoder(categorical_features='all', dtype=<type 'float'>,
       n_values='auto', sparse=True)

In [28]:
train_psc

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,DepartmentDescriptionInt
1,22,7,5,60538815980,1,SHOES,8932,62
2,22,7,5,7410811099,1,PERSONAL CARE,4505,50
3,18,8,5,2238403510,2,PAINT AND ACCESSORIES,3566,49
4,18,8,5,2006613744,2,PAINT AND ACCESSORIES,1018,49
5,18,8,5,2006618783,2,PAINT AND ACCESSORIES,1018,49
6,18,8,5,2006613743,1,PAINT AND ACCESSORIES,1018,49
7,18,8,5,7004802737,1,PAINT AND ACCESSORIES,2803,49
8,18,8,5,2238495318,1,PAINT AND ACCESSORIES,4502,49
10,18,8,5,5200010239,1,DSD GROCERY,4607,17
11,18,8,5,88679300501,2,PAINT AND ACCESSORIES,3505,49


In [29]:
a = enc.transform(train_psc.TripType.reshape(-1, 1)).toarray()

flmap = {}
flnd = {}

ddmax = np.max(train.DepartmentDescriptionInt) + 1

fnum = 6
# 0 - % items returned
# 1 - 1 item
# 2 - 2 items
# 3 - 3-4 items
# 4 - 5-9 items
# 5 - 10+ items

f_ddstart = 6
f_ddend = 6 + ddmax

# 6-ddmax: coarse dept description
fnum += ddmax 

# rest of fnum: fineline mapping
for cat in range(0, ddmax):
#for cat in [20]:
    catmask = train_psc.DepartmentDescriptionInt == cat
    subset = train_psc[catmask]
    
    #print(cat, len(subset))
    if len(subset) < 10:
        continue
        
#    vc = subset.Upc.value_counts()
    vc = subset.FinelineNumber.value_counts()
    
    for iterit in vc.iteritems():
        fln = iterit[0]
        if (iterit[1] < 10): # or (len(subset) < (iterit[1] * 2)):
            continue
            
        fnum += 1
                
        flmap[(cat, fln)] = fnum
        
print(len(flmap.keys()))
for i in flmap:
    print(i, flmap[i])

6119
((32, 8246), 3003)
((24, 3601), 1910)
((60, 8002), 5552)
((31, 2018), 2928)
((35, 4111), 3297)
((17, 2505), 1342)
((25, 9805), 2373)
((43, 8538), 3993)
((3, 6707), 273)
((5, 4661), 489)
((13, 2286), 895)
((57, 1263), 5266)
((49, 3553), 4286)
((23, 257), 1906)
((17, 3664), 1340)
((62, 3232), 5805)
((25, 4221), 2285)
((8, 655), 664)
((24, 3031), 2034)
((19, 535), 1635)
((24, 3307), 1943)
((43, 1081), 3837)
((66, 1402), 6032)
((17, 4642), 1293)
((13, 3762), 871)
((19, 1057), 1577)
((24, 2805), 1944)
((41, 6567), 3703)
((15, 3001), 924)
((30, 3811), 2720)
((12, 90), 800)
((52, 1119), 4741)
((23, 3458), 1833)
((24, 2404), 2122)
((31, 223), 2893)
((24, 2451), 2071)
((35, 6604), 3299)
((41, 6504), 3718)
((27, 556), 2487)
((8, 2951), 612)
((17, 8301), 1304)
((23, 1356), 1826)
((17, 4206), 1441)
((4, 4903), 320)
((64, 2255), 5959)
((29, 7903), 2643)
((15, 2429), 999)
((19, 8054), 1580)
((5, 2231), 379)
((39, 4023), 3536)
((29, 8048), 2658)
((21, 1908), 1754)
((11, 222), 714)
((16, 1348), 1

In [30]:

def make_ddcomb(df, num_visits = 100000000):
    
    num_ents = len(df)
    
    visits = np.sort(np.unique(df.VisitNumber))
    
    num_visits = min(num_visits, len(visits))

    ddmax = np.max(train.DepartmentDescriptionInt)

    mat = np.zeros((num_visits, (ddmax * 1) + fnum + 2))
    tt = np.zeros(num_visits)
 
    df_scancount = df.ScanCount.values
    df_visitnumber = df.VisitNumber.values
    df_triptype = df.TripType.values if ('TripType' in df) else np.zeros(len(df))
    df_ddint = df.DepartmentDescriptionInt.values
    df_fln = df.Upc.values
    df_fln = df.FinelineNumber.values
    #    df_fmap = df.fmap.values
    df_weekday = df.Weekday.values
    
    
    visitmap = {}
    
    vnum = -1
    
    icount = np.zeros(num_visits + 1)
    
    for i in range(0, num_ents):
        try:
            visit = visitmap[df_visitnumber[i]]
        except:
            vnum += 1

            if (vnum + 1) == num_visits:
                break
            
            visitmap[df_visitnumber[i]] = vnum
            visit = vnum
            
            tt[vnum] = df_triptype[i]
#            mat[visit][fnum + 1] = df_weekday[i] >= 6
#            mat[visit][fnum + 2] = df_weekday[i] == 1
#            mat[visit][fnum + df_weekday[i]] = 1
            
        icount[visit] += 1
    
        if True: #df_scancount[i] > 0:
            dept = df_ddint[i]
            fln = df_fln[i]
            
            # allocation space: 
            # 0 - % returns
            # 1 - ddmax - regular map
            # ddmax - ddmax*2 - leftovers
            # ddmax*2 >= +fmap - map
            
            mat[visit][6 + df_ddint[i]] += (1 + ((df_scancount[i] - 1) * .25))
            
            try:
                feature = flmap[(dept, fln)]
                mat[visit][feature] += (1 + ((df_scancount[i] - 1) * .25))
            except:
                None
                #feature = 1 + ddmax + dept
                #mat[visit][feature] += (1 + ((df_scancount[i] - 1) * .25))
                
        if df_scancount[i] < 0:
            mat[visit][0] += 1
            
    vnum += 1
    for i in range(0, vnum):
        if np.sum(mat[i][f_ddstart:f_ddend]):
#            mat[i][] /= np.sum(mat[i][f_ddstart:f_ddmax])
            mat[i][f_ddstart:f_ddend] /= np.sum(mat[i][f_ddstart:f_ddend])
        if np.sum(mat[i][f_ddend:fnum]):
            mat[i][f_ddend:fnum] /= np.sum(mat[i][f_ddend:fnum])
        if icount[i] > 0:
            mat[i][0] /= icount[i]
            
            if icount[i] == 1:
                mat[i][1] = 1
            elif icount[i] == 2:
                mat[i][2] = 1
            elif icount[i] < 5:
                mat[i][3] = 1
            elif icount[i] < 10:
                mat[i][4] = 1
            else:
                mat[i][5] = 1
                
            
    return mat, visits, tt

In [31]:
num_ents =len(train)

In [32]:
num_ents

630913

In [33]:
visits = np.sort(np.unique(train.VisitNumber))

In [34]:
visits

array([     5,      7,      8, ..., 191345, 191346, 191347])

In [35]:
num_visits = min(100000000, len(visits))

In [36]:
num_visits

93326

In [37]:
ddmax = np.max(train.DepartmentDescriptionInt)

In [38]:
ddmax

68

In [39]:
fnum

6194

In [40]:
mat = np.zeros((num_visits, (ddmax * 1) + fnum + 2))

In [49]:
mat.shape

(93326, 6264)

In [42]:
tt = np.zeros(num_visits)
tt

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [43]:
df_scancount = train.ScanCount.values

In [44]:
df_visitnumber = train.VisitNumber.values
df_triptype = train.TripType.values if ('TripType' in train) else np.zeros(len(train))
df_ddint = train.DepartmentDescriptionInt.values
df_fln = train.Upc.values
df_fln = train.FinelineNumber.values
#    df_fmap = df.fmap.values
df_weekday = train.Weekday.values

In [45]:
visitmap = {}
vnum = -1
icount = np.zeros(num_visits + 1)

In [86]:
icount

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [48]:
for i in range(0, 50):
    try:
        visit = visitmap[df_visitnumber[i]]
        print visit,'visit1'
    except:
        vnum += 1
        print vnum, 'vnum'
        if (vnum + 1) == num_visits:
            break
            
        visitmap[df_visitnumber[i]] = vnum
        print visitmap[df_visitnumber[i]],'visitmap'
        visit = vnum
        print visit, 'visit2'
           
        tt[vnum] = df_triptype[i]
#            mat[visit][fnum + 1] = df_weekday[i] >= 6
#            mat[visit][fnum + 2] = df_weekday[i] == 1
#            mat[visit][fnum + df_weekday[i]] = 1
            
        icount[visit] += 1
    
    if True: #df_scancount[i] > 0:
        dept = df_ddint[i]
        print dept,'dept'
        fln = df_fln[i]
        print fln, 'fln'
            
            # allocation space: 
            # 0 - % returns
            # 1 - ddmax - regular map
            # ddmax - ddmax*2 - leftovers
            # ddmax*2 >= +fmap - map
            
        mat[visit][6 + df_ddint[i]] += (1 + ((df_scancount[i] - 1) * .25))
        print mat,mat.shape,'mat2'
            
        try:
            feature = flmap[(dept, fln)]
            print feature, 'feature'
            mat[visit][feature] += (1 + ((df_scancount[i] - 1) * .25))
        except:
            None
                #feature = 1 + ddmax + dept
                #mat[visit][feature] += (1 + ((df_scancount[i] - 1) * .25))
                
    if df_scancount[i] < 0:
        mat[visit][0] += 1

0 visit1
20 dept
1001 fln
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 2.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] (93326, 6264) mat2
1699 feature
1 visit1
62 dept
8932 fln
[[ 2.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 2.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] (93326, 6264) mat2
1 visit1
50 dept
4505 fln
[[ 2.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 2.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] (93326, 6264) mat2
4492 feature
2 visit1
49 dept
3566 fln
[[ 2.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 2.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] (93326, 6264)

In [31]:
train_ddmat, train_visitnum, train_tt = make_ddcomb(train)
#train_ddmat, train_visitnum, train_tt = make_ddcomb(train,num_visits=5000)

In [33]:
train_tt = pd.DataFrame(train_tt)
for i in train_tt.columns:
    print i

0


In [34]:
train_visitnum = pd.DataFrame(train_visitnum)
for i in train_visitnum.columns:
    print i

0


In [35]:
train_ddmat = pd.DataFrame(train_ddmat)
for i in train_ddmat.columns:
    print i

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [36]:
train_ddmat

,0,1,2,3,4,5,6,7,8,9,...,6254,6255,6256,6257,6258,6259,6260,6261,6262,6263
0,1.000000,1,0,0,0,0,0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0,1,0,0,0,0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,0.086957,0,0,0,0,1,0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0,0,1,0,0,0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,0,0,1,0,0,0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
5,0.000000,0,0,1,0,0,0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
6,0.000000,0,0,0,1,0,0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
7,0.000000,0,0,0,1,0,0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
8,0.000000,0,0,1,0,0,0,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
9,0.000000,0,0,0,1,0,0,0.111111,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
valid_ddmat, valid_visitnum, valid_tt = make_ddcomb(valid)
#test_ddmat, test_visitnum, test_tt = make_ddcomb(test)

In [ ]:
n = 16000
n = len(train_tt)
dtrain = xgb.DMatrix(train_ddmat[0:n], label=train_tt[0:n].tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 8,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.35,
          'colsample_bytree': 1,
          'eta': 0.1}

watchlist = [(dtrain, 'train'), (dtest, 'eval')]
#watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bstc = xgb.train(xgb_params, dtrain, 2000, evals=watchlist, verbose_eval=False, early_stopping_rounds=25)
#bst = xgb.train(xgb_params, dtrain, 10)

predictv2 = bstc.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predictv2))

# .707850 for categorized counts :) (may be even lower)
# .707369 712 - 1.0 ss, 0.2 cat
# .717141 - 0.35/1.0

In [ ]:

dtrain = xgb.DMatrix(train_ddmat, label=train_tt.tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

bst_cb = {}
predict_cb = {}
#for subs in [0.2, 0.35, 0.5, 0.65, 0.8, 0.9, 1.0]: #np.arange(0.1, 0.9, 0.1):
#    for cb in [0.2, 0.4, 0.6, 0.8, 0.9, 1.0]:
def dorunrun(subs, cb):
        xgb_params = {'max_depth': 8,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': subs,
          'colsample_bytree': cb,
          'eta': 0.01}

        watchlist = [(dtrain, 'train'), (dtest, 'eval')]
        bst_cb[(subs, cb)] = xgb.train(xgb_params, dtrain, 10000, evals=watchlist, verbose_eval=False, early_stopping_rounds=50)

        predict_cb[(subs, cb)] = bst_cb[(subs, cb)].predict(xgb.DMatrix(valid_ddmat))
        print(subs, cb, multiclass_log_loss(valid_tt, predict_cb[(subs, cb)]))

dorunrun(0.35, 0.8)
dorunrun(1.0, 0.2)

In [ ]:
n = 4000
n = len(train_tt)
dtrain = xgb.DMatrix(train_ddmat[0:n], label=train_tt[0:n].tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 20,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.3,
          'colsample_bytree': 0.75,
          'eta': 0.01}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(xgb_params, dtrain, 4000) #, evals=watchlist, verbose_eval=True)
#bst = xgb.train(xgb_params, dtrain, 10)

predictv = bst.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predictv))

# 1.22593422041 - reference "fast" run w/4000
# 1.21145138814 - new ref fast, 80 feat

# .9320 lowest w/24 md
# .927077 w/20
# .927421 w/18



In [ ]:
dtrain = xgb.DMatrix(train_ddmat, label=train_tt.tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 64,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.3,
          'colsample_bytree': 0.8,
          'eta': 0.01}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bst10 = xgb.train(xgb_params, dtrain, 1250, evals=watchlist, verbose_eval=True)
#bst = xgb.train(xgb_params, dtrain, 10)

#predict10 = bst10.predict(xgb.DMatrix(valid_ddmat))
#print(multiclass_log_loss(valid_tt, predict10))

#predict = bst10.predict(xgb.DMatrix(train_ddmat))
#print(multiclass_log_loss(train_tt[0:len(train_ddmat)], predict))

# [1024]	eval-mlogloss:1.249856	train-mlogloss:0.456847

# [938]	eval-mlogloss:1.287829	train-mlogloss:0.475465 - fr .985
# [855]	eval-mlogloss:1.306764	train-mlogloss:0.503785 - fr .95

# [850]	eval-mlogloss:1.256834	train-mlogloss:0.522326 - fr .975 on base

# [942]	eval-mlogloss:1.253929	train-mlogloss:0.474049 .99 on base

# [300]	eval-mlogloss:1.263669	train-mlogloss:1.064013 - regular
# [293]	eval-mlogloss:1.263084	train-mlogloss:1.071790 - .99 cosine distance check, 57features
# [1249]	eval-mlogloss:0.980208	train-mlogloss:0.611616 - ibid

'''
with returns in one column:

+=2:
[269]	eval-mlogloss:1.262641	train-mlogloss:1.079731
[500]	eval-mlogloss:1.060215	train-mlogloss:0.809933
[1151]	eval-mlogloss:0.966263	train-mlogloss:0.611675

+=1
[270]	eval-mlogloss:1.261297	train-mlogloss:1.079624
[500]	eval-mlogloss:1.057744	train-mlogloss:0.810384
[1132]	eval-mlogloss:0.966242	train-mlogloss:0.614787
[1249]	eval-mlogloss:0.963500	train-mlogloss:0.598182

short run:
[1068]	eval-mlogloss:1.257743	train-mlogloss:0.455562
'''

In [ ]:
dtrain = xgb.DMatrix(train_ddmat, label=train_tt.tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 12,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.3,
          'colsample_bytree': 0.75,
          'eta': 0.01}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(xgb_params, dtrain, 2000) #, evals=watchlist, verbose_eval=True)
#bst = xgb.train(xgb_params, dtrain, 10)
predictv = bst.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predictv))


In [ ]:
dtrain = xgb.DMatrix(train_ddmat, label=train_tt.tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 12,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.3,
          'colsample_bytree': 0.75,
          'eta': 0.01}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(xgb_params, dtrain, 1450) #, evals=watchlist, verbose_eval=True)
#bst = xgb.train(xgb_params, dtrain, 10)

predictv = bst.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predictv))


In [ ]:
for n in range(500,len(train_tt),500):
    dtrain = xgb.DMatrix(train_ddmat[0:n], label=train_tt[0:n].tolist())
    dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

    xgb_params = {'max_depth': 12,
            'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.3,
          'colsample_bytree': 0.75,
          'eta': 0.05}

    watchlist = [(dtest, 'eval'), (dtrain, 'train')]
    bst = xgb.train(xgb_params, dtrain, 300) #, evals=watchlist, verbose_eval=True)
    #bst = xgb.train(xgb_params, dtrain, 10)

    predictv = bst.predict(xgb.DMatrix(valid_ddmat))
    print(n, multiclass_log_loss(valid_tt, predictv))


In [ ]:
dtrain = xgb.DMatrix(train_ddmat[0:4000], label=train_tt[0:4000].tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 10,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.3,
          'colsample_bytree': 0.75,
          'eta': 0.05}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(xgb_params, dtrain, 500, evals=watchlist, verbose_eval=True)
#bst = xgb.train(xgb_params, dtrain, 10)

#predictv = bst.predict(xgb.DMatrix(valid_ddmat))
#print(multiclass_log_loss(valid_tt, predictv))


In [ ]:
dtrain = xgb.DMatrix(train_ddmat, label=train_tt.tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 12,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.3,
          'colsample_bytree': 0.8,
          'eta': 0.01}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bst10 = xgb.train(xgb_params, dtrain, 1000, evals=watchlist, verbose_eval=True)
#bst = xgb.train(xgb_params, dtrain, 10)

predictv = bst10.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predict11))


In [ ]:
dtrain = xgb.DMatrix(train_ddmat, label=train_tt[0:len(train_ddmat)].tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 11,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'eta': 0.01}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bst11 = xgb.train(xgb_params, dtrain, 1250, evals=watchlist, verbose_eval=False)
#bst = xgb.train(xgb_params, dtrain, 10)

predict11 = bst11.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predict11))

predict = bst.predict(xgb.DMatrix(train_ddmat))
print(multiclass_log_loss(train_tt[0:len(train_ddmat)], predict))

In [ ]:
dtrain = xgb.DMatrix(train_ddmat, label=train_tt[0:len(train_ddmat)].tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 10,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'seed': 12345,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'eta': 0.01}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bst10a = xgb.train(xgb_params, dtrain, 1250, evals=watchlist, verbose_eval=False)
#bst = xgb.train(xgb_params, dtrain, 10)

predict10a = bst10a.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predict10a))

predict = bst.predict(xgb.DMatrix(train_ddmat))
print(multiclass_log_loss(train_tt[0:len(train_ddmat)], predict))

In [ ]:
test_ddmat, test_visitnum, test_tt = make_ddcomb(test)

In [ ]:
n = 4000
n = len(train_tt)
dtrain = xgb.DMatrix(train_ddmat[0:n], label=train_tt[0:n].tolist())
dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

xgb_params = {'max_depth': 20,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.3,
          'colsample_bytree': 0.75,
          'eta': 0.01}

watchlist = [(dtest, 'eval')]
bst = xgb.train(xgb_params, dtrain, 4000, evals=watchlist, verbose_eval=False, early_stopping_rounds=25)
#bst = xgb.train(xgb_params, dtrain, 10)

predictv = bst.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predictv))

In [ ]:
test_ddmat, test_visitnum, test_tt = make_ddcomb(test)
predict = bst_a[7].predict(xgb.DMatrix(test_ddmat))

subkeys = {}

for k in sub.loc[0].keys():
    if 'TripType_' in k:
        subkeys[k] = int(k[9:])
        
        if subkeys[k] == 999:
            subkeys[k] = 45
            
sub = sub_orig.copy()

predict_t = predict.transpose()
for k in subkeys.keys():
    pk = triptype_map[subkeys[k]]
    sub[k] = predict_t[pk]
    
with gzip.open('xgb-718.gz', 'wt') as write_file:
    sub.to_csv(write_file, index=False)

In [ ]:

for i in range(len(predictv)):
    print(i, valid_tt[i], predictv[i][valid_tt[i]])

In [ ]:
predictv[656]

In [ ]:
dtrain = xgb.DMatrix(predictv[100:], label=valid_tt[100:].tolist())
dtest = xgb.DMatrix(predictv[0:100], label=valid_tt[0:100].tolist())

xgb_params = {'max_depth': 6,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'seed': 12345,
          'subsample': 0.25,
          'colsample_bytree': 1.0,
          'eta': 0.01}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bstl2 = xgb.train(xgb_params, dtrain, 1250, evals=watchlist, verbose_eval=True)
#bst = xgb.train(xgb_params, dtrain, 10)

In [ ]:
np.max(train.DepartmentDescriptionInt)

In [ ]:
bst_a = {}
for d in range(6,9):
    n = len(train_tt)
    dtrain = xgb.DMatrix(train_ddmat[0:n], label=train_tt[0:n].tolist())
    dtest = xgb.DMatrix(valid_ddmat, label=valid_tt.tolist())

    xgb_params = {'max_depth': d,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.5,
          'colsample_bytree': 0.8,
          'eta': 0.01}

    watchlist = [(dtest, 'eval')]
    bst_a[d] = xgb.train(xgb_params, dtrain, 10000, evals=watchlist, verbose_eval=False, early_stopping_rounds=100)

    try:
        predictv = bst_a[d].predict(xgb.DMatrix(valid_ddmat))
        print(d, multiclass_log_loss(valid_tt, predictv))

#        bst_a[d].save_model('dmp{0}.dmp'.format(d))
    except:
        None

In [ ]:
predict_v = {}
for k in bst_a.keys():
    predict_v[k] = bst_a[k].predict(xgb.DMatrix(valid_ddmat))
    print(k, multiclass_log_loss(valid_tt, predict_v[k]))

In [ ]:
s = [6, 7]

count = 0
p = None
for i in s:
    count += 1
    try:
        p += predict_v[i].copy()
    except:
        p = predict_v[i].copy()
    print(i, multiclass_log_loss(valid_tt, p / count))

In [ ]:
multiclass_log_loss(valid_tt, predict_v[7])

In [ ]:
predictv = bsta.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predictv))


In [ ]:
tm = []
for x in flmap.keys():
    tm.append(x[1])

In [ ]:
len(np.unique(tm))

In [ ]:
len(flmap.keys())

In [ ]:
bst_b = bst_a.copy()

In [ ]:
px = predict_v[7].transpose()

In [ ]:
px = px.copy()
import sklearn.metrics

In [ ]:
tt = np.zeros(len(valid_tt))
for i in range(px.shape[0]):
    for j in range(px.shape[1]):
        tt[j] = valid_tt[j] == i
        
    if np.sum(tt) > 25:
        print(i, np.sum(tt), sklearn.metrics.roc_auc_score(tt, px[i]), sklearn.metrics.log_loss(tt, px[i]))

In [ ]:
pc = predict_v[7].copy()
for i in range(0, len(valid_tt)):
#    pc[i][31] = (valid_tt[i] == 31)
#    pc[i][35] = (valid_tt[i] == 35)
#    pc[i][6] = (valid_tt[i] == 6)
    pc[i] /= np.sum(pc[i])

In [ ]:
multiclass_log_loss(valid_tt, pc)

In [ ]:
n = len(train_tt)

key = 31

ttt_key = np.zeros(n)
vtt_key = np.zeros(len(valid_tt))

for i in range(n):
    ttt_key[i] = 1 if (train_tt[i] == key) else 0
    
for i in range(len(valid_tt)):
    vtt_key[i] = 1 if (valid_tt[i] == key) else 0

dtrain = xgb.DMatrix(train_ddmat[0:n], label=ttt_key)
dtest = xgb.DMatrix(valid_ddmat, label=vtt_key)

xgb_params = {'max_depth': 12,
          'objective': 'reg:linear',
          'eval_metric': 'logloss',
#          'num_class': 1,
          'subsample': 0.4,
          'colsample_bytree': 0.8,
          'eta': 0.01}

watchlist = [(dtrain, 'train'), (dtest, 'eval')]
#watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bsta = xgb.train(xgb_params, dtrain, 5000, evals=watchlist, verbose_eval=True, early_stopping_rounds=500)
#bst = xgb.train(xgb_params, dtrain, 10)

In [ ]:
huh = bsta.predict(xgb.DMatrix(valid_ddmat))
huh[huh < 0] = 0
huh[huh > 1] = 1

In [ ]:
huh

In [ ]:
pc = predict_v[7].copy()
print(multiclass_log_loss(valid_tt, pc))
for i in range(0, len(valid_tt)):
    pc[i][31] = huh[i]
#    pc[i][31] = (valid_tt[i] == 31)
#    pc[i][35] = (valid_tt[i] == 35)
#    pc[i][6] = (valid_tt[i] == 6)
    pc[i] /= np.sum(pc[i])
print(multiclass_log_loss(valid_tt, pc))

In [ ]:
predict_t = bst_a[7].predict(xgb.DMatrix(train_ddmat))

In [ ]:
pickle.dump(predict_v[7], file=open('predictv-718.pkl', 'wb'))

In [ ]:
import pickle

In [ ]:
pickle.dump(predict, file=open('predict-6813.pkl', 'wb'))

In [ ]:
len(train_psc[train_psc.DepartmentDescriptionInt == 0]) / len(train_psc)

In [ ]:
len(train_psc) / len(train)

In [ ]:
print(multiclass_log_loss(valid_tt, predictv))


In [ ]:
predictv = bstb.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predictv))


In [ ]:
predictvc = bstc.predict(xgb.DMatrix(valid_ddmat))
print(multiclass_log_loss(valid_tt, predictvc))

In [ ]:
print(multiclass_log_loss(valid_tt, (predictvc+predictv)/2))

In [ ]:
len(train.Upc.unique())

In [ ]:
len(train.FinelineNumber.unique())

In [ ]:
5188*5188

In [ ]:
nfl = len(train.FinelineNumber.unique())
flgrid = np.zeros((38, nfl, nfl))

In [ ]:
flgrid.shape

In [ ]:
flnum = 0
flremap = {}
for i in np.unique(train.FinelineNumber):
    flremap[i] = flnum
    flnum += 1

In [ ]:
for i in range(0, 38):
    subset = train[train.TripType == i]
    print(i, len(subset))
    
    for v in np.unique(subset.VisitNumber):
        subset2 = subset[subset.VisitNumber == v]
        
        for m in range(len(subset2)):
            ma = flremap[subset2.iloc[m].FinelineNumber]
            for n in range(m + 1, len(subset2)):
                na = flremap[subset2.iloc[n].FinelineNumber]
                flgrid[i][ma][na] += 1
#                print(m, ma, na, flgrid[i][ma][na])

In [ ]:
flgrid[0]

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
np.sum(flgrid[32])

In [ ]:
from scipy import sparse

In [ ]:
csr[0] = sparse.csr_matrix(flgrid[0])

In [ ]:
csr = {}

In [ ]:
for i in range(0,38):
    csr[i] = sparse.csr_matrix(flgrid[i])

In [ ]:
import pickle

In [ ]:
pickle.dump(csr, file=open('/run/shm/csr.pkl', 'wb'))

In [ ]:
csr[0]

In [ ]:
aflgrid = np.zeros((nfl, nfl))
for i in range(0, 38):
    aflgrid += flgrid[i]

In [ ]:
vr = np.zeros((len(np.unique(valid.VisitNumber)), 38))
c = 0

for v in np.unique(valid.VisitNumber):
    subset = valid[valid.VisitNumber == v]
    if (len(subset) < 2):
        vr[c][32] = 1
    else:
        for m in range(len(subset)):
            try:
                ma = flremap[subset.iloc[m].FinelineNumber]

                for n in range(m + 1, len(subset)):
                    try:
                        na = flremap[subset.iloc[n].FinelineNumber]
                    
                        for g in range(0, 38):
                            vr[c][g] += np.log(flgrid[g][ma][na])
                    except:
                        None
            except:
                None
                
    if (np.sum(vr[c])) > 0:
        vr[c] /= np.sum(vr[c])

    c += 1
    
    if not (c % 100):
        print(c)

        
    
        

In [ ]:
np.sum(vr[1])

In [ ]:
print(multiclass_log_loss(valid_tt, vr))


In [ ]:
pkl = []
for k in predict_cb.keys():
    pkl.append(k)
    
for m in range(0, len(pkl)):
    print(pkl[m], multiclass_log_loss(valid_tt, predict_cb[pkl[m]]))
    for n in range(m + 1, len(pkl)):
            v = multiclass_log_loss(valid_tt, (predict_cb[pkl[m]] + predict_cb[pkl[n]] / 2))
            if v < 0.705:
                print(pkl[m], pkl[n], multiclass_log_loss(valid_tt, predict_cb[pkl[n]]), v)

In [ ]:
pa = predict_cb[(1.0, 0.2)]
pb = predict_cb[(0.35, 0.8)]

In [ ]:
for i in np.arange(0.1, 2, 0.1):
    print(i, multiclass_log_loss(valid_tt, ((pa * i) + pb) / (1 + i)))

In [ ]:
pm = ((pa * 1.4) + pb) / 2.4

In [ ]:
import pickle
pickle.dump(pm, file=open('predict-6814.pkl', 'wb'))

In [ ]:
pa = predict_cb[(1.0, 0.2)]
pb = predict_cb[(0.35, 0.8)]

test_ddmat, test_visitnum, test_tt = make_ddcomb(test)
predicta = bst_cb[(1.0, 0.2)].predict(xgb.DMatrix(test_ddmat))
predictb = bst_cb[(0.35, 0.8)].predict(xgb.DMatrix(test_ddmat))
predict = ((predicta * 1.4) + predictb) / 2.4
subkeys = {}

for k in sub.loc[0].keys():
    if 'TripType_' in k:
        subkeys[k] = int(k[9:])
        
        if subkeys[k] == 999:
            subkeys[k] = 45
            
sub = sub_orig.copy()

predict_t = predict.transpose()
for k in subkeys.keys():
    pk = triptype_map[subkeys[k]]
    sub[k] = predict_t[pk]
    
with gzip.open('xgb-718.gz', 'wt') as write_file:
    sub.to_csv(write_file, index=False)